In [1]:
import pprint
import sys
import pickle
from tester import dump_classifier_and_data
import sys
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import RandomizedPCA
from time import time
import pylab as pl
from feature_format import featureFormat
from feature_format import targetFeatureSplit
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA


/Users/Haleh/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# %load poi_id.py
#!/usr/bin/python
sys.path.append("../tools/")

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

## Task 1: Select what features you'll use.
features_list is a list of strings, each of which is a feature name.<br>
The first feature must be "poi".<br>
You will need to use more features

In [3]:
### Task 1: Select what features you'll use.

features_list = ['poi',
                 'salary',
                 'deferral_payments',
                 'total_payments',
                 'loan_advances',
                 'bonus',
                 'restricted_stock_deferred',
                 'bonus_salary_R',
                 'restricted_stock_deferred',
                 'deferred_income',
                 'total_stock_value',
                 'expenses',
                 'exercised_stock_options',
                 'other',
                 'long_term_incentive',
                 'restricted_stock',
                 'director_fees',
                 'to_messages',
                 'from_poi_to_this_person',
                 'from_poi_to_this_person_Percent',                 
                 'from_messages',
                 'from_this_person_to_poi',
                 'from_this_person_to_poi_percent',
                 'shared_receipt_with_poi']


## Task 2: Remove outliers
based on mini-project in outlier lessons, i know there is a key "Total", which holds the total value for all other values. <br>
Another one is the key "THE TRAVEL AGENCY IN THE PARK".<br>
I'm going to exclude both of them



In [4]:
names=[]
for k, v in data_dict.iteritems():
    names.append( k)
pprint.pprint (names)
pprint.pprint (data_dict["TOTAL"], width=1)
pprint.pprint (data_dict["THE TRAVEL AGENCY IN THE PARK"])

['METTS MARK',
 'BAXTER JOHN C',
 'ELLIOTT STEVEN',
 'CORDES WILLIAM R',
 'HANNON KEVIN P',
 'MORDAUNT KRISTINA M',
 'MEYER ROCKFORD G',
 'MCMAHON JEFFREY',
 'HORTON STANLEY C',
 'PIPER GREGORY F',
 'HUMPHREY GENE E',
 'UMANOFF ADAM S',
 'BLACHMAN JEREMY M',
 'SUNDE MARTIN',
 'GIBBS DANA R',
 'LOWRY CHARLES P',
 'COLWELL WESLEY',
 'MULLER MARK S',
 'JACKSON CHARLENE R',
 'WESTFAHL RICHARD K',
 'WALTERS GARETH W',
 'WALLS JR ROBERT H',
 'KITCHEN LOUISE',
 'CHAN RONNIE',
 'BELFER ROBERT',
 'SHANKMAN JEFFREY A',
 'WODRASKA JOHN',
 'BERGSIEKER RICHARD P',
 'URQUHART JOHN A',
 'BIBI PHILIPPE A',
 'RIEKER PAULA H',
 'WHALEY DAVID A',
 'BECK SALLY W',
 'HAUG DAVID L',
 'ECHOLS JOHN B',
 'MENDELSOHN JOHN',
 'HICKERSON GARY J',
 'CLINE KENNETH W',
 'LEWIS RICHARD',
 'HAYES ROBERT E',
 'MCCARTY DANNY J',
 'KOPPER MICHAEL J',
 'LEFF DANIEL P',
 'LAVORATO JOHN J',
 'BERBERIAN DAVID',
 'DETMERING TIMOTHY J',
 'WAKEHAM JOHN',
 'POWERS WILLIAM',
 'GOLD JOSEPH',
 'BANNANTINE JAMES M',
 'DUNCAN JOHN H'

In [5]:
data_dict.pop("TOTAL")
data_dict.pop("THE TRAVEL AGENCY IN THE PARK")

{'bonus': 'NaN',
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'NaN',
 'exercised_stock_options': 'NaN',
 'expenses': 'NaN',
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 362096,
 'poi': False,
 'restricted_stock': 'NaN',
 'restricted_stock_deferred': 'NaN',
 'salary': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 362096,
 'total_stock_value': 'NaN'}

# Task 3: Create new feature(s)
Store to my_dataset for easy export below.

In [6]:
for key, value in data_dict.iteritems():
    if value['bonus'] == "NaN" or value['salary'] == "Nan":
        value['bonus_salary_R'] = "NaN"
    else:
        value['bonus_salary_R'] = float(value['bonus']) / float(value['salary'])

for key, value in data_dict.iteritems():
    if value["from_this_person_to_poi"] == "NaN" or value["to_messages"] == "Nan":
        value['from_this_person_to_poi_percent'] = "NaN"
    else:
        value['from_this_person_to_poi_percent'] = float(value['from_this_person_to_poi'])*100 / float(value['to_messages'])

for key, value in data_dict.iteritems():
    if value["from_poi_to_this_person"] == "NaN" or value["to_messages"] == "Nan":
        value['from_poi_to_this_person_Percent'] = "NaN"
    else:
        value['from_poi_to_this_person_Percent'] = float(value['from_poi_to_this_person'])*100 / float(value['to_messages'])
        


In [7]:
my_dataset = data_dict
### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

In [8]:
df = pd.DataFrame.from_dict(data_dict, orient = 'index')
df = df[features_list]
df = df.replace('NaN', np.nan)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 144 entries, ALLEN PHILLIP K to YEAP SOON
Data columns (total 24 columns):
poi                                144 non-null bool
salary                             94 non-null float64
deferral_payments                  38 non-null float64
total_payments                     123 non-null float64
loan_advances                      3 non-null float64
bonus                              81 non-null float64
restricted_stock_deferred          17 non-null float64
bonus_salary_R                     81 non-null float64
restricted_stock_deferred          17 non-null float64
deferred_income                    48 non-null float64
total_stock_value                  125 non-null float64
expenses                           94 non-null float64
exercised_stock_options            101 non-null float64
other                              91 non-null float64
long_term_incentive                65 non-null float64
restricted_stock                   109 non-null float64

# Task 4: Try a varity of classifiers
Please name your classifier clf for easy export below.
Note that if you want to do PCA or other multi-stage operations,
you'll need to use Pipelines. For more info:
http://scikit-learn.org/stable/modules/pipeline.html

In [9]:
# Provided to give you a starting point. Try a variety of classifiers.

data = featureFormat(data_dict, features_list)
labels, features = targetFeatureSplit(data)
features_train, features_test, labels_train, labels_test = train_test_split(features, labels,  test_size=0.3, random_state=42)


# Data Prep

## Feature Scaling 

In [23]:
#minmax scaler
scaler = MinMaxScaler()
scaled_feature = scaler.fit_transform(features_train)

## Feature Decompositoin


In [17]:
pca = PCA()
pca.fit(features_train)
features_train = pca.transform(features_train)
features_test = pca.transform(features_test)

## Feature selection

In [26]:
#selectKbest
select = SelectKBest()
best_features = SelectKBest(k= 5 ).fit(features_train, labels_train)
features_train_selected = best_features.transform(features_train)
features_test_selected = best_features.transform(features_test)

In [ ]:
#selectpercentile
from sklearn.feature_selection import SelectPercentile
selector = SelectPercentile(percentile=20)
selector.fit(features_train, labels_train)
features_train_selected = selector.transform(features_train).toarray()
features_test_selected  = selector.transform(features_test).toarray()

# Algorithms

## Classification 

In [19]:
#Naive Bayes
Gaussian = GaussianNB()
clf = Gaussian.fit(features_train, labels_train) 
pred = clf.predict(features_test)
acc = accuracy_score(pred, labels_test)
print "accuracy:", acc
print metrics.recall_score(labels_test, pred)
print metrics.precision_score(labels_test, pred)

accuracy: 0.511627906977
0.428571428571
0.15


In [ ]:
#Support Vector Machine (SVM)
svc = SVC(kernel="linear") 
# svc = SVC(kernel="rbf", C = , gamma = )
clf = svc.fit(features_train, labels_train) 
pred = clf.predict(features_test)
acc = accuracy_score(pred, labels_test)
print metrics.recall_score(labels_test, pred)
print metrics.precision_score(labels_test, pred)

In [21]:
# Decision Trees
dtc = DecisionTreeClassifier(splitter='random', random_state=42) 
clf = dtc.fit(features_train, labels_train) 
pred = clf.predict(features_test)
acc = accuracy_score(pred, labels_test)
print "accuracy:", acc
print metrics.recall_score(labels_test, pred)
print metrics.precision_score(labels_test, pred)


accuracy: 0.837209302326
0.571428571429
0.5


In [18]:
#K Nearest Neighbors
knc = KNeighborsClassifier()
clf = knc.fit(features_train, labels_train) 
pred = clf.predict(features_test)
acc = accuracy_score(pred, labels_test)
print "accuracy:", acc
print metrics.recall_score(labels_test, pred)
print metrics.precision_score(labels_test, pred)

accuracy: 0.837209302326
0.142857142857
0.5


# Validation

## KFold

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits= ... , shuffle=True )
kf.get_n_splits(X)
print(kf)

## GridSearchCV

In [ ]:
from sklearn import svm 
from sklearn.model_selection import GridSearchCV
# adjust:
# parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
# clf = grid_search.GridSearchCV(svr, parameters)
# clf.fit(features, labels)

In [15]:

### Task 5: Tune your classifier to achieve better than .3 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)



In [16]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)